POSSIBILI CLASSIFICATORI

implementazione dei possibili classificatori che prendono in input l'espressione genica e vanno a classificare il tipo di tumore. Per fare questo si andranno ad utilizzare diversi tipi di classificatori in particolare: SVM, random forest, un multilayer perceptron, Bagging, Gradient Boos e una Deep neural network

SVM con diversi huperparametri

In [1]:
from sklearn.model_selection import train_test_split

#andiamo a caricare i dati 

dataset="inserire il dataset etichettato"
labels='sono le etichette'

#ricordarsi di inserire il seme per rendere il tutto ripetibile

#divisione per andare a prendere il test
X_train,Y_train,X_test,Y_test=train_test_split(dataset,labels,test_size=0.3,random_state=0)
#divisione per andare a prendere i valori anche per il validation
X_train,Y_train,X_val,Y_val=train_test_split(X_train,Y_train,test_size=0.3,random_state=0)

ValueError: Found input variables with inconsistent numbers of samples: [31, 17]

In [2]:

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split

#andiamo a dividere il dataset 


hyperparameters = {
    "kernel": ["linear","poly","rbf","sigmoid"],
    "decision_function_shape":['ovr','ovo'],
    "degree":[2,3]
}

param_grid = list(ParameterGrid(hyperparameters))

results_df = pd.DataFrame(
    columns=["Kernel", "Decision_function_shape","degree", "Accuracy", "Precision", "Recall", "F1 Score"]
)

for par in param_grid:
    #definiamo il modello con tutti i  possibili iperparametri
    svm_model = SVC(**par)
    svm_model.fit(X_train,Y_train)

    #andiamo a valutare il modello
    
    y_val_pred=svm_model.predict(X_val)
    
    
    #andiamo a definire le possibili statistiche
    f1_macro=f1_score(Y_val,y_val_pred,avarage='macro')
    Accuracy=accuracy_score(Y_val,y_val_pred)
    precision=precision_score(Y_val,y_val_pred)
    recall=recall_score(Y_val,y_val_pred)

    #andiamo a mettere i vari valori nel dataframe
    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "Kernel": par["Kernel"],
                    "Decision_function_shape": par["decision_function_shape"],
                    "F1 Score": f1_macro,
                    'precision':precision,
                    'recall':recall,
                    'accuracy':Accuracy
                },
                index=[0],
            ),
        ],
        ignore_index=True,
    )

results_df.to_csv('Risultati_svm.csv')

NameError: name 'X_train' is not defined

Adesso vado a provare l'approccio con Random Forest

In [ ]:

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split


hyperparameters = {
    "estimator":[100,200,500],
    "criterion":['gini', 'entropy', 'log_loss'],
    "max_depth": [6,8,10],
    "min_samples_split":[4,6,8],
    "n_jobs":[4]
}

param_grid = list(ParameterGrid(hyperparameters))

results_df = pd.DataFrame(
    columns=["estimator", "criterion","max_depth","min_samples_split", "Accuracy", "Precision", "Recall", "F1 Score"]
)

#non è detto che serva questa parte da valutare mentre si prova
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


for par in param_grid:
    
    # Train the Random Forest model
    rf_model = RandomForestClassifier(**par)
    rf_model.fit(X_train, Y_train)

    # Make predictions
    y_val_pred = rf_model.predict(X_val)

    #andiamo a definire le possibili statistiche
    f1_macro=f1_score(Y_val,y_val_pred,avarage='macro')
    Accuracy=accuracy_score(Y_val,y_val_pred)
    precision=precision_score(Y_val,y_val_pred)
    recall=recall_score(Y_val,y_val_pred)

    #andiamo a mettere i vari valori nel dataframe
    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "estimator":par['estimator'],
                    "criterion":par['criterion'],
                    "max_depth":par["max_depth":],
                    "min_samples_split":par["min_samples_split"],
                    "F1 Score": f1_macro,
                    'precision':precision,
                    'recall':recall,
                    'accuracy':Accuracy
                },
                index=[0],
            ),
        ],
        ignore_index=True,
    )

results_df.to_csv('Risultati_RandomForest.csv')

Adesso vado a provare l'implementazione di Bagging. Sempre con diversi hyperparametri

In [ ]:


import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

#si vanno a definire diversi modelli base per la classificazione in particloare
#logistic regressio, random forest e multi layer perceptron

lr = LogisticRegression()
rdf= RandomForestClassifier()
mlp= MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)


#si definiscono gli hyperparametri 

hyperparameters = {
    "base_estimator":[lr,rdf,mlp],
    "warm_start":[True, False],
    'n_estimator':[100,200],
    "n_jobs":[4]
}

param_grid = list(ParameterGrid(hyperparameters))

results_df = pd.DataFrame(
    columns=["base_estimator", "warm_start",'n_estimator', "Accuracy", "Precision", "Recall", "F1 Score"]
)

#non è detto che serva questa parte da valutare mentre si prova
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Addestrare il modello Bagging
for par in param_grid:

    #deffiniamo il modello e dopodichè andiamo a fare il training 
    bagging_model = BaggingClassifier(**par)
    bagging_model.fit(X_train, Y_train)

    #valutiamo adesso il modello
    y_val_pred=bagging_model.predict(X_val)

    #calcolo delle statistiche
    f1_macro=f1_score(Y_val,y_val_pred,avarage='macro')
    Accuracy=accuracy_score(Y_val,y_val_pred)
    precision=precision_score(Y_val,y_val_pred)
    recall=recall_score(Y_val,y_val_pred)


    #andiamo a mettere i vari valori nel dataframe
    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "base_estimator":par['base_estimator'],
                    "warm_start":par['warm_start'],
                    "n_estimator":par["n_estimator":],
                    "F1 Score": f1_macro,
                    'precision':precision,
                    'recall':recall,
                    'accuracy':Accuracy
                },
                index=[0],
            ),
        ],
        ignore_index=True,
    )

results_df.to_csv('Risultati_Bagging.csv')

Adesso si prova GradientBoostingClassifier

In [ ]:

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier



#si vanno a definire diversi modelli base per la classificazione in particloare
#logistic regressio, random forest e multi layer perceptron

lr = LogisticRegression()
rdf= RandomForestClassifier()
mlp= MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

#si definiscono gli hyperparametri 
hyperparameters = {
    "base_estimator":[lr,rdf,mlp],
    'n_estimator':[100,200],
}

param_grid = list(ParameterGrid(hyperparameters))

results_df = pd.DataFrame(
    columns=["base_estimator",'n_estimator', "Accuracy", "Precision", "Recall", "F1 Score"]
)

#non è detto che serva questa parte da valutare mentre si prova
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


for par in param_grid:
    
    #si definisce il modello e poi si procede con la fit
    gb_model = GradientBoostingClassifier(**par)
    gb_model.fit(X_train, Y_train)

    #si calcolano le predizione
    y_val_pred=gb_model.predict(Y_val)


    #si calcolano le statistiche 
    f1_macro=f1_score(Y_val,y_val_pred,avarage='macro')
    Accuracy=accuracy_score(Y_val,y_val_pred)
    precision=precision_score(Y_val,y_val_pred)
    recall=recall_score(Y_val,y_val_pred)


    #andiamo a mettere i vari valori nel dataframe
    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "base_estimator":par['base_estimator'],
                    "warm_start":par['warm_start'],
                    "n_estimator":par["n_estimator":],
                    "F1 Score": f1_macro,
                    'precision':precision,
                    'recall':recall,
                    'accuracy':Accuracy
                },
                index=[0],
            ),
        ],
        ignore_index=True,
    )

results_df.to_csv('Risultati_GradientBoostingClassifier.csv')

Adesso si prova il MultiLayerPerceptro

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier



#si definiscono gli hyperparametri 
hyperparameters = {
    "alpha":[0.0001,0.001,0.01,0.1],
    'learning_rate':['constant','adaptive'],
    'solver':['adam','sgd','lbfgs']
}

param_grid = list(ParameterGrid(hyperparameters))

results_df = pd.DataFrame(
    columns=["alpha",'learning_rate','solver', "Accuracy", "Precision", "Recall", "F1 Score"]
)

#non è detto che serva questa parte da valutare mentre si prova
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


for par in param_grid:

    #vado a creare il modello e vado poi a fare la fit

    mlp_model = MLPClassifier(**par)
    mlp_model.fit(X_train, Y_train)

    #vado adesso a valutare il modello

    y_val_pred=mlp_model.predict(Y_val)

    #vado a calcolare le statistiche

    f1_macro=f1_score(Y_val,y_val_pred,avarage='macro')
    Accuracy=accuracy_score(Y_val,y_val_pred)
    precision=precision_score(Y_val,y_val_pred)
    recall=recall_score(Y_val,y_val_pred)


    #andiamo a mettere i vari valori nel dataframe
    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "alpha":par['alpha'],
                    "learning_rate":par['learning_rate'],
                    "solver":par["solver":],
                    "F1 Score": f1_macro,
                    'precision':precision,
                    'recall':recall,
                    'accuracy':Accuracy
                },
                index=[0],
            ),
        ],
        ignore_index=True,
    )

results_df.to_csv('Risultati_MultiLayerPerceptron.csv')

Implementazione della Deep Neural Network